### Imports

In [1]:
import pandas as pd
import numpy as np

import sys
from PIL import Image
sys.modules["Image"] = Image # might not need this or import sys?

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize



### Load data

In [49]:
cd /Users/localadmin/Documents/BloomTech/Unit4/U4_Kaggle/leaf-classification

/Users/localadmin/Documents/BloomTech/Unit4/U4_Kaggle/leaf-classification


In [51]:
X_test = pd.read_csv("test.csv", index_col="id")
train = pd.read_csv("train.csv", index_col="id")

In [45]:
print(len(X_test))
X_test.shape[0] + train.shape[0]

594


1584

In [6]:
train.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [52]:
X_train = train.drop(["species"], axis=1)
y_train_text = train["species"]

X_train.shape
# There are 192 columns, so input dim = 192

y_train_text.nunique()
# there are 99 possible species

99

In [47]:
# encode the categories
# there's probably a better way to do this—see CNN example from Ryan
categories = list(set(y_train_text))
numbers = list(range(100))
categories_dict = {key: value for key, value in zip(categories, numbers)}
categories_dict
categories_lookup = {key: value for key, value in zip(numbers, categories)}

y_train_nums = [categories_dict[cat] for cat in y_train_text]
y_train = pd.Series(y_train_nums)
y_train.shape

(990,)

### Build Simple MLP model

In [61]:
input_dim = X_train.shape[1] # 192 columns

output_dim = y_train.nunique() # 99 possible species

# instantiate model
mlp = Sequential()

# input layer, hidden layer 1
mlp.add(Dense(128, activation="relu", input_dim=input_dim))

# hidden layer 2
mlp.add(Dense(64, activation="relu"))

# hidden layer 3
mlp.add(Dense(32, activation="relu"))

# output layer
# QUESTION: Will having an output layer with so many more nodes than previous
#           hidden layer cause problems??
mlp.add(Dense(output_dim, activation="softmax"))

# Compile model
mlp.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


In [62]:
mlp.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               24704     
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 99)                3267      
                                                                 
Total params: 38,307
Trainable params: 38,307
Non-trainable params: 0
_________________________________________________________________


In [64]:
mlp.fit(X_train, y_train, epochs=100)

Epoch 1/100
31/31 [==============================] - 0s 2ms/step - loss: 2.9747 - accuracy: 0.2364
Epoch 2/100
31/31 [==============================] - 0s 2ms/step - loss: 2.5333 - accuracy: 0.3606
Epoch 3/100
31/31 [==============================] - 0s 2ms/step - loss: 2.2406 - accuracy: 0.4040
Epoch 4/100
31/31 [==============================] - 0s 2ms/step - loss: 1.9745 - accuracy: 0.4909
Epoch 5/100
31/31 [==============================] - 0s 2ms/step - loss: 1.7684 - accuracy: 0.5242
Epoch 6/100
31/31 [==============================] - 0s 2ms/step - loss: 1.5871 - accuracy: 0.5707
Epoch 7/100
31/31 [==============================] - 0s 2ms/step - loss: 1.4495 - accuracy: 0.5970
Epoch 8/100
31/31 [==============================] - 0s 2ms/step - loss: 1.3131 - accuracy: 0.6434
Epoch 9/100
31/31 [==============================] - 0s 2ms/step - loss: 1.2097 - accuracy: 0.6687
Epoch 10/100
31/31 [==============================] - 0s 2ms/step - loss: 1.1275 - accuracy: 0.6727
Epoch 11/

In [89]:
mlp_predictions_train = mlp.predict(X_train)
print("Predicted value:", mlp_predictions[1].argmax())
print("Predicted value text name:", categories_lookup[43])
print("Actual value:", list(y_train_text)[1])


Predicted value: 43
Predicted value text name: Pterocarya_Stenoptera
Actual value: Pterocarya_Stenoptera


In [152]:
mlp_predictions = pd.DataFrame(mlp.predict(X_test))
mlp_predictions
mlp_predictions[id] = pd.Series(X_test.index)
id_num = mlp_predictions.columns[-1]

In [153]:
# clean up DF for submission
# rename columns
mlp_predictions.rename(columns = categories_lookup, inplace=True)
mlp_predictions.rename(columns = {id_num: "id"}, inplace=True) # this number will need to update every time cell above is run
reordered_cols = list(mlp_predictions.columns[:-1])
reordered_cols.sort()
reordered_cols.insert(0, "id")

# reorder columns
mlp_predictions = mlp_predictions[reordered_cols]
# mlp_predictions.set_index("id", inplace=True)
mlp_predictions

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,1.118933e-16,1.590961e-09,1.454150e-10,6.791619e-03,1.229343e-16,1.227668e-12,1.402886e-07,1.586424e-20,1.069750e-27,...,4.894187e-25,2.793591e-13,5.972178e-15,3.655168e-13,1.296074e-07,6.744242e-09,1.132148e-31,2.133546e-13,1.245910e-22,3.400233e-04
1,7,5.884440e-22,3.949949e-10,4.111775e-13,1.628219e-04,2.640916e-22,5.045759e-12,4.169566e-07,1.369826e-13,2.186002e-28,...,1.789420e-25,8.698296e-11,3.642006e-19,3.767931e-11,7.609919e-11,1.032522e-08,1.841464e-29,8.193821e-13,5.482810e-29,9.228056e-05
2,9,1.860679e-10,9.976450e-01,2.398865e-23,2.607763e-21,2.946706e-08,3.171818e-21,2.443626e-07,3.003624e-10,1.254204e-03,...,1.035279e-12,0.000000e+00,2.828355e-15,5.331231e-14,1.582504e-08,9.673375e-06,4.782857e-08,6.739525e-37,1.817542e-28,1.632443e-07
3,12,6.590655e-15,7.086346e-08,1.522102e-37,4.651010e-23,6.312881e-23,1.001675e-28,2.490619e-11,1.058382e-08,1.412550e-07,...,1.978069e-19,0.000000e+00,3.121297e-19,4.176964e-19,1.511512e-08,1.806250e-05,4.749197e-12,0.000000e+00,0.000000e+00,4.226427e-11
4,13,5.502218e-16,5.836231e-11,0.000000e+00,1.536021e-33,5.657486e-16,0.000000e+00,6.010914e-14,1.118879e-11,2.776786e-05,...,2.974053e-10,0.000000e+00,3.086631e-15,1.199080e-21,1.027768e-07,3.875430e-09,6.789959e-05,0.000000e+00,1.193016e-37,5.746331e-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,1576,1.931138e-08,9.931022e-01,4.523056e-22,5.042412e-21,1.448089e-06,1.124080e-21,7.247174e-08,1.642099e-11,2.799352e-03,...,3.445417e-12,0.000000e+00,7.269413e-14,1.393854e-13,9.673921e-08,8.038164e-06,8.393059e-08,8.407707e-37,4.516392e-26,3.038783e-07
590,1577,2.062046e-08,4.904870e-07,1.002122e-28,2.188452e-12,7.748977e-19,1.548414e-28,2.449171e-11,4.103984e-07,5.822599e-11,...,5.703563e-13,2.829591e-29,2.312028e-10,1.725610e-12,7.146195e-05,1.259185e-03,3.609889e-12,2.404692e-26,3.186969e-30,3.802074e-04
591,1579,1.196703e-11,4.806598e-10,2.064313e-10,1.575716e-07,1.546544e-14,2.684944e-19,1.595581e-15,8.476014e-22,1.145650e-25,...,4.170884e-26,3.210259e-12,8.657367e-17,3.034326e-12,4.895642e-11,2.152397e-13,2.671330e-31,3.271411e-15,5.614624e-23,2.494454e-04
592,1580,2.696292e-28,8.019719e-15,2.642987e-09,3.783090e-08,5.756794e-20,9.002170e-15,2.557590e-08,3.607410e-10,3.065872e-31,...,1.546407e-15,2.179338e-13,1.793875e-17,3.053934e-03,4.367874e-13,8.252235e-13,2.502355e-19,4.339525e-13,1.352669e-15,1.091041e-10


In [155]:
# export to a csv, omitting the index
mlp_predictions.to_csv("submission_basic_mlp.csv", index=False)

### Build a CNN model

#### Get images into workable format for CNN modeling

In [9]:
cd images

/Users/localadmin/Documents/BloomTech/Unit4/U4_Kaggle/leaf-classification/images


In [10]:
# load images
raw_images = [load_img("{}.jpg".format(i)) for i in range(1, 1585)]

In [11]:
# resize images so they all are the same size
resized = [smart_resize(img, (200,200)) for img in raw_images]

In [53]:
# get an array for each image
image_arrays_all = []
for img in resized:
    arr = img_to_array(img)
    image_arrays_all.append(arr)

# get the training/test ids to filter for training arrays
train_ids = list(X_train.index)
test_ids = list(X_test.index)

# get just the training images:
X_train = np.array([image_arrays_all[i] for i in range(1584) if i+1 in train_ids])

# get just the test images:
X_test = np.array([image_arrays_all[i] for i in range(1584) if i+1 in test_ids])

# Now have lists of 500x500x3 arrays to feed into a CNN


#### Build CNN model

In [13]:
# Setting up some of the values to be used in CNN model

n_features = y_train.nunique()

n_weight_matrices = 128 # 128 is pulled from U4S3M2 notebook, not entirely sure where it comes from?

weight_matrix_size = (3, 3)

image_dim = (200, 200, 3)

pool_size = (2, 2)

In [15]:
# instantiate Sequential model
cnn = Sequential()

# 1st convolutional layer
cnn.add(Conv2D(n_weight_matrices,
               weight_matrix_size,
               activation="relu",
               input_shape=image_dim)) # not doing padding here because images are large enough

# 1st pooling layer
cnn.add(MaxPooling2D(pool_size))

# 2nd convolutional layer
cnn.add(Conv2D(n_weight_matrices/2,
               weight_matrix_size,
               activation="relu"))

# 2nd pooling layer
cnn.add(MaxPooling2D(pool_size))

# flatten the image matrix into a row vector, in order to pass to Dense layer
cnn.add(Flatten())

# Dense layer
cnn.add(Dense(1000, activation="relu")) # 200 here is a little arbitrary

# Output layer
cnn.add(Dense(n_features,
              activation="softmax"))



In [16]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 147456)            0         
                                                                 
 dense (Dense)               (None, 1000)              1

In [17]:
# compile model
cnn.compile(optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

In [18]:
cnn.fit(X_train,
        y_train,
        epochs=10)

Epoch 1/10
31/31 [==============================] - 128s 4s/step - loss: 154.0899 - accuracy: 0.0848
Epoch 2/10
31/31 [==============================] - 126s 4s/step - loss: 2.1491 - accuracy: 0.5303
Epoch 3/10
31/31 [==============================] - 153s 5s/step - loss: 0.4107 - accuracy: 0.8990
Epoch 4/10
31/31 [==============================] - 154s 5s/step - loss: 0.1318 - accuracy: 0.9697
Epoch 5/10
31/31 [==============================] - 170s 5s/step - loss: 0.1021 - accuracy: 0.9778
Epoch 6/10
31/31 [==============================] - 158s 5s/step - loss: 0.0464 - accuracy: 0.9949
Epoch 7/10
31/31 [==============================] - 157s 5s/step - loss: 0.0101 - accuracy: 1.0000
Epoch 8/10
31/31 [==============================] - 160s 5s/step - loss: 0.0035 - accuracy: 1.0000
Epoch 9/10
31/31 [==============================] - 188s 6s/step - loss: 7.9263e-04 - accuracy: 1.0000
Epoch 10/10
31/31 [==============================] - 181s 6s/step - loss: 4.2980e-04 - accuracy: 1.0000

#### Get predictions, prep submission

In [34]:
cnn_predictions_train = cnn.predict(X_train)


In [35]:
img_to_predict = 3
predicted_value = cnn_predictions_train[img_to_predict].argmax()
print("Predicted value:", predicted_value)
print("Predicted value text name:", categories_lookup[predicted_value])
print("Actual value:", list(y_train_text)[img_to_predict])

Predicted value: 33
Predicted value text name: Tilia_Tomentosa
Actual value: Tilia_Tomentosa


In [56]:
cnn_predictions = pd.DataFrame(cnn.predict(X_test))
cnn_predictions
cnn_predictions[id] = pd.Series(test_ids)
id_num = cnn_predictions.columns[-1]

In [57]:
# clean up DF for submission
# rename columns
cnn_predictions.rename(columns = categories_lookup, inplace=True)
cnn_predictions.rename(columns = {id_num: "id"}, inplace=True) # this number will need to update every time cell above is run
reordered_cols = list(cnn_predictions.columns[:-1])
reordered_cols.sort()
reordered_cols.insert(0, "id")

# reorder columns
cnn_predictions = cnn_predictions[reordered_cols]
# cnn_predictions.set_index("id", inplace=True)
cnn_predictions

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,8.106623e-05,1.259686e-06,4.497655e-06,6.357399e-07,4.371473e-07,1.064271e-05,2.210948e-05,1.787252e-07,2.169914e-06,...,6.532019e-05,1.421809e-05,1.589366e-01,2.602568e-04,8.721631e-05,9.544155e-04,7.672620e-05,3.049799e-01,2.489771e-05,3.112633e-05
1,7,5.744773e-05,1.795802e-07,2.316865e-05,2.780673e-05,5.804315e-05,8.297911e-08,4.828428e-06,1.666686e-06,3.428636e-08,...,4.131335e-04,1.177282e-04,3.194118e-06,5.039605e-06,1.548521e-07,4.847025e-08,2.084728e-07,1.137595e-06,1.832174e-04,2.339785e-04
2,9,4.912823e-09,3.780112e-04,6.797048e-05,5.086963e-05,8.114126e-07,8.321194e-12,5.890025e-06,1.407325e-05,9.893792e-04,...,1.930091e-06,9.480527e-08,8.047171e-08,3.518421e-07,2.137698e-10,7.504079e-08,1.339812e-08,6.433827e-08,2.899309e-10,1.104447e-07
3,12,2.057996e-05,2.997638e-11,1.040833e-09,1.932854e-10,6.039242e-09,9.562216e-13,2.274199e-10,3.638096e-11,8.260496e-12,...,1.593299e-09,1.145884e-09,2.595158e-09,1.266687e-12,1.196049e-09,1.664459e-07,7.317711e-08,4.887784e-11,4.628161e-07,5.789470e-08
4,13,8.199739e-09,2.450934e-11,1.351598e-13,1.415005e-12,2.997965e-13,1.129949e-13,1.284232e-10,7.191007e-12,5.315812e-13,...,1.892333e-10,4.126071e-12,9.945427e-01,1.062371e-10,6.359919e-11,1.005112e-05,2.916746e-12,4.446171e-06,8.573233e-12,2.349722e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,1576,5.508543e-08,9.925072e-01,8.977300e-07,6.820263e-03,8.745443e-06,1.803680e-06,1.360545e-04,4.272387e-08,5.959734e-08,...,1.584549e-08,4.498761e-09,7.605556e-11,2.097481e-12,7.006529e-10,3.194781e-10,1.901933e-13,6.389896e-09,7.982116e-11,1.010675e-10
590,1577,1.621944e-13,5.366108e-14,1.303459e-11,7.172350e-14,3.427653e-14,5.736127e-13,2.862403e-11,1.726746e-15,2.943537e-11,...,3.647899e-13,3.145417e-13,9.848018e-08,8.171928e-12,7.913447e-13,9.760912e-08,3.603600e-12,6.606497e-10,8.628449e-11,1.205594e-11
591,1579,1.477300e-08,3.094329e-08,6.690258e-07,3.849647e-08,9.266618e-08,1.820620e-09,1.681905e-06,1.632293e-08,4.067323e-09,...,2.174886e-02,1.766755e-05,1.056988e-07,6.581613e-09,3.424189e-08,1.690586e-07,1.907064e-05,5.872527e-09,1.557745e-06,5.786420e-05
592,1580,1.205542e-05,1.032837e-05,1.513496e-05,3.162950e-06,2.186334e-06,3.190827e-06,3.068970e-05,3.532315e-07,4.538256e-07,...,3.631963e-05,4.524484e-05,6.309218e-04,1.647816e-06,1.262635e-06,4.913752e-06,3.759089e-06,3.543491e-03,3.115393e-06,1.445883e-05


In [58]:
cd /Users/localadmin/Documents/BloomTech/Unit4/U4_Kaggle/submissions

/Users/localadmin/Documents/BloomTech/Unit4/U4_Kaggle/submissions


In [59]:
# export to a csv, omitting the index
cnn_predictions.to_csv("submission_basic_cnn.csv", index=False)